In [9]:
# Dependencies 
from bs4 import BeautifulSoup as bs
import requests
import pymongo
from splinter import Browser
from selenium import webdriver
import pandas as pd
import time
import pprint

In [2]:
!which chromedriver

/usr/local/bin/chromedriver


In [3]:
executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)

# NASA Mars News

In [4]:
# Get url to scape
url= 'https://mars.nasa.gov/news/'
browser.visit(url)
html = browser.html

# Create BeautifulSoup object; parse with 'html.parser'
soup = bs(html, 'html.parser')

In [5]:
# Collect the latest news title and paragraph
article = soup.find("div", class_='list_text')
title = article.find("div", class_="content_title").text
p = article.find("div", class_ ="article_teaser_body").text

# Print title and paragraph
print(title)
print('------------------------')
print(p)

NASA Readies Perseverance Mars Rover's Earthly Twin 
------------------------
Did you know NASA's next Mars rover has a nearly identical sibling on Earth for testing? Even better, it's about to roll for the first time through a replica Martian landscape.


# JPL Mars Space Images - Featured Image

In [7]:
# URL for featured image 
image_url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'

browser.visit(image_url)

# Go to full image
browser.click_link_by_partial_text('FULL IMAGE')
time.sleep(5)

# Go to more info
browser.click_link_by_partial_text('more info')

# Create BeautifulSoup object; parse with 'html.parser'
html = browser.html
image_soup = bs(html, 'html.parser')

# Scrape the URL
img_url = image_soup.find('figure', class_='lede').a['href']
featured_image_url = f'https://www.jpl.nasa.gov{img_url}'
print(featured_image_url)

https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA17011_hires.jpg


# Mars Facts

In [18]:
# Visit Mars Facts webpage
facts_url = "https://space-facts.com/mars/"
browser.visit(facts_url)
html = browser.html

In [20]:
# Scrape the table containing facts about Mars
table = pd.read_html(facts_url)
mars_facts = table[2]

# Rename columns
mars_facts.columns = ['Description','Value']

# Reset Index to be description
mars_facts.set_index('Description', inplace=True)
mars_facts

,Value
Description,
Equatorial Diameter:,"6,792 km"
Polar Diameter:,"6,752 km"
Mass:,6.39 × 10^23 kg (0.11 Earths)
Moons:,2 (Phobos & Deimos)
Orbit Distance:,"227,943,824 km (1.38 AU)"
Orbit Period:,687 days (1.9 years)
Surface Temperature:,-87 to -5 °C
First Record:,2nd millennium BC
Recorded By:,Egyptian astronomers


In [21]:
# Use Pandas to convert the data to a HTML table string
mars_facts.to_html('table.html')

# Mars Hemispheres

In [22]:
# Visit the USGS Astrogeology site
hemi_url = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"
browser.visit(hemi_url)

# Create BeautifulSoup object; parse with 'html.parser'
html = browser.html
hemi_soup = bs(html, "html.parser")

# Create list to hold titles & links to images
hemisphere_image_urls = []

# Retrieve all elements that contain image information
results = hemi_soup.find("div", class_ = "result-list" )
hemispheres = results.find_all("div", class_="item")

# Loop through each image
for hemisphere in hemispheres:
    hemi_title = hemisphere.find("h3").text
    hemi_title = hemi_title.replace("Enhanced", "")
    end_url = hemisphere.find("a")["href"]
    img_url = "https://astrogeology.usgs.gov/" + end_url    
    browser.visit(img_url)
    html = browser.html
    soup = bs(html, "html.parser")
    downloads = soup.find("div", class_="downloads")
    image_url = downloads.find("a")["href"]
    hemisphere_image_urls.append({"title": hemi_title, "img_url": image_url})

# Print image title and url
pp = pprint.PrettyPrinter()
pp.pprint(hemisphere_image_urls)

[{'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg',
  'title': 'Cerberus Hemisphere '},
 {'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg',
  'title': 'Schiaparelli Hemisphere '},
 {'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg',
  'title': 'Syrtis Major Hemisphere '},
 {'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg',
  'title': 'Valles Marineris Hemisphere '}]
